In [ ]:
from manim import *
#from classes.christmas_fourier_classes import *
from PIL import Image
import jupyter_capture_output

video_scene = f" -v WARNING --disable_caching sw_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s sw_Scene"

In [2]:
GLOBAL_AMPLITUDE = 1

# incoming wave
def get_wave_in(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    def wave_in(x):
        return np.sin(k_wave*x - omega_wave*t)
    return wave_in


# reflected wave
def get_wave_out(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    def wave_out(x):
        return np.sin(-k_wave*x - omega_wave*t - PI)
    return wave_out


# superpositioned wave
def get_super_wave(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    def super_wave(x):
        return 0
    return 0

In [ ]:
%%capture_video --path "animations/standing_wave/standing_wave.mp4"
%%manim -qm --fps 60 $video_scene


class cf_Scene(Scene):
    def construct(self):
        self.camera.background_color = WHITE